#Student Name:
#Student ID:

#Student Name:
#Student ID:

#Student Name:
#Student ID:

#Student Name:
#Student ID:

In [15]:
import numpy as np
import pandas as pd
import re

In [19]:
# Following code is from https://github.com/amclean3695/Communities-and-Crime/blob/master/Communities_and_Crime.ipynb
# TODO : See is there any other means for pulling column names from C45 file format
#Reads in names from textfile and performs regex to extract the attribute names 
textfile = open('communities.names','r')
filetext = textfile.read()
textfile.close()
matches = re.findall(r'-{2}\s(\w+)\:{1}', filetext)
col_names = matches[2:] #corresponds to the 128 attributes

In [29]:
df = pd.read_csv('communities.data', header=None, names=col_names)

#data_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data'
#df = pd.read_csv(data_url, header=None)
# The above code does not work as PythonAnywhere blocks outgoing connections for free accounts to non-whitelisted sites

#print(df.head(5))
#dir(df)
print(df.shape)
#print(df['ViolentCrimesPerPop'].value_counts())

(1994, 128)


### Remove Non-Predictive Attributes

Of the 128 attributes loaded, for this dataset it is stated that:

Attribute Information: (122 predictive, 5 non-predictive, 1 goal) 
* state: US state (by number) - not counted as predictive above, but if considered, should be consided nominal (nominal) 
* county: numeric code for county - not predictive, and many missing values (numeric) 
* community: numeric code for community - not predictive and many missing values (numeric) 
* communityname: community name - not predictive - for information only (string) 
* fold: fold number for non-random 10 fold cross validation, potentially useful for debugging, paired tests - not predictive (numeric) 

Therefore, the first five columns shall be removed from the dataset

In [30]:
df = df.drop(['state', 'county', 'community', 'communityname', 'fold'], axis=1)
print(df.shape)


(1994, 123)


In [35]:
#print(df.head(5))
# From DePy_Talk Noteboook
# How much of your data is missing?
df.isnull().sum().sort_values(ascending=False).head()

ViolentCrimesPerPop    0
PctNotHSGrad           0
PctUnemployed          0
PctEmploy              0
PctEmplManu            0
dtype: int64

There are no cells with missing data
However, visually inspecting the data, there are cells with "?" and this needs to be replaced with "NaN" (Not a Number)

In [38]:
df = df.replace('?', np.nan)
print(df.columns[df.isnull().any()])

Index(['OtherPerCap', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps',
       'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop',
       'PolicReqPerOffic', 'PolicPerPop', 'RacialMatchCommPol',
       'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian',
       'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz',
       'PolicAveOTWorked', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr',
       'LemasGangUnitDeploy', 'PolicBudgPerPop'],
      dtype='object')


In [60]:
df.isnull().sum().sort_values(ascending=False).head(25)

OtherPerCap            1
PctOccupManu           0
HispPerCap             0
NumUnderPov            0
PctPopUnderPov         0
PctLess9thGrade        0
PctNotHSGrad           0
PctBSorMore            0
PctUnemployed          0
PctEmploy              0
PctEmplManu            0
PctEmplProfServ        0
ViolentCrimesPerPop    0
NumIlleg               0
MalePctDivorce         0
MalePctNevMarr         0
FemalePctDiv           0
TotalPctDiv            0
PersPerFam             0
PctFam2Par             0
PctKids2Par            0
PctYoungKids2Par       0
PctTeen2Par            0
PctWorkMomYoungKids    0
PctOccupMgmtProf       0
dtype: int64

In [58]:
#print(df.isnull().sum())
print(df.shape)
print(df.shape[0])
print(df['LemasSwFTFieldPerPop'].isnull().sum())
for column in df:
    if(df[column].isnull().sum()>=(df.shape[0]/2)):
        print("Column ", column, df[column].isnull().sum())
        df = df.drop([column], axis=1)
print(df.shape)

(1994, 123)
1994
1675
Column  LemasSwornFT 1675
Column  LemasSwFTPerPop 1675
Column  LemasSwFTFieldOps 1675
Column  LemasSwFTFieldPerPop 1675
Column  LemasTotalReq 1675
Column  LemasTotReqPerPop 1675
Column  PolicReqPerOffic 1675
Column  PolicPerPop 1675
Column  RacialMatchCommPol 1675
Column  PctPolicWhite 1675
Column  PctPolicBlack 1675
Column  PctPolicHisp 1675
Column  PctPolicAsian 1675
Column  PctPolicMinor 1675
Column  OfficAssgnDrugUnits 1675
Column  NumKindsDrugsSeiz 1675
Column  PolicAveOTWorked 1675
Column  PolicCars 1675
Column  PolicOperBudg 1675
Column  LemasPctPolicOnPatr 1675
Column  LemasGangUnitDeploy 1675
Column  PolicBudgPerPop 1675
(1994, 101)


In [59]:
df.isnull().sum().sort_values(ascending=False).head(25)

OtherPerCap            1
PctOccupManu           0
HispPerCap             0
NumUnderPov            0
PctPopUnderPov         0
PctLess9thGrade        0
PctNotHSGrad           0
PctBSorMore            0
PctUnemployed          0
PctEmploy              0
PctEmplManu            0
PctEmplProfServ        0
ViolentCrimesPerPop    0
NumIlleg               0
MalePctDivorce         0
MalePctNevMarr         0
FemalePctDiv           0
TotalPctDiv            0
PersPerFam             0
PctFam2Par             0
PctKids2Par            0
PctYoungKids2Par       0
PctTeen2Par            0
PctWorkMomYoungKids    0
PctOccupMgmtProf       0
dtype: int64

There is only one missing value remaining but lets impute it for the craic using code from the DePy_Talk notebook